In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
tasks = [
    "facility location",
    "image summarization",
    "movie recommendation",
    "revenue maximization"
]

For each application, we sample 4 points from (r, b)-space. For each point, we construct 4 problem instances and compare the result of two greedy algorithms: density greedy and marginal gain greedy.
So it is better we show our results in a table.

Specification: 
for each task, we create a csv file with filename set task name.
In each csv file, columns: cost ratio, budget, DG performance, MGG performance, is consistent

Once these four data files are well-established, we convert them into latex table.

Definition of 4 points from (r, b)-space.
Write a script to search for these 4 points. 

# Post Data Analysis

In [7]:
df = pd.read_csv("../result/CostInterpolation-20250207/test/output_500.txt")
df.head()

,model_name,cost_ratio,budget,density,marginal_gain,is_dominance_theoretical,is_dominance_practical
0,movie,1.1,2.0,7.317861,7.317861,True,True
1,movie,1.1,3.0,13.198527,13.376736,True,False
2,movie,1.1,3.2,18.611354,18.442580,True,True
3,movie,1.1,4.2,22.625186,18.722640,True,True
4,movie,1.1,4.4,22.950489,23.148300,True,False


In [12]:
def compute_coverage(data_frame, model_name=None):
    if model_name is not None:
        # compute model specific coverage
        data_frame = data_frame[data_frame["model_name"] == model_name]
    coverage = len(data_frame[data_frame["is_dominance_theoretical"] == data_frame["is_dominance_practical"]]) / len(data_frame)
    return coverage                       

def check_coverage(path: str="../result/CostInterpolation-20250207/test/output_500.txt"):
    df = pd.read_csv(path)
    total_coverage = compute_coverage(df)
    print("Total coverage is {}".format(total_coverage))
    for model_name in ["movie", "image"]:
        model_specific_coverage = compute_coverage(df, model_name)
        print("Model specific coverage for {} is {}".format(model_name, model_specific_coverage))

check_coverage("../result/CostInterpolation-20250207/test/output_500.txt")
check_coverage("../result/CostInterpolation-20250207/test/output_1000.txt")

Total coverage is 0.3238095238095238
Model specific coverage for movie is 0.48095238095238096
Model specific coverage for image is 0.16666666666666666
Total coverage is 0.3238095238095238
Model specific coverage for movie is 0.48095238095238096
Model specific coverage for image is 0.16666666666666666
